<a href="https://colab.research.google.com/github/leonpalafox/colab_notebooks/blob/main/TokenizerCount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PDF Token Counter for Google Colab
# This script counts tokens in PDF files using various tokenizers from Hugging Face

import os
import sys
import argparse
from typing import Dict, List, Optional
from google.colab import files

# PDF processing
import PyPDF2

# Hugging Face tokenizers
from transformers import (
    AutoTokenizer,
    GPT2Tokenizer,
    T5Tokenizer,
    BertTokenizer,
    RobertaTokenizer
)

def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extract all text from a PDF file

    Args:
        pdf_path: Path to the PDF file

    Returns:
        Extracted text as a string
    """
    text = ""

    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)

            print(f"PDF has {num_pages} pages")

            for page_num in range(num_pages):
                page = reader.pages[page_num]
                text += page.extract_text() + "\n"

        return text

    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return ""

def count_tokens(text: str, tokenizer_name: str) -> int:
    """
    Count tokens in text using the specified tokenizer

    Args:
        text: Text to tokenize
        tokenizer_name: Name of the tokenizer to use

    Returns:
        Number of tokens
    """
    tokenizers = {
        "gpt2": GPT2Tokenizer.from_pretrained("gpt2"),
        "t5": T5Tokenizer.from_pretrained("t5-base"),
        "bert": BertTokenizer.from_pretrained("bert-base-uncased"),
        "roberta": RobertaTokenizer.from_pretrained("roberta-base"),
    }

    # Add support for custom tokenizers from Hugging Face
    if tokenizer_name not in tokenizers:
        try:
            tokenizers[tokenizer_name] = AutoTokenizer.from_pretrained(tokenizer_name)
        except Exception as e:
            print(f"Error loading tokenizer {tokenizer_name}: {e}")
            return -1

    tokenizer = tokenizers[tokenizer_name]
    tokens = tokenizer.encode(text)

    return len(tokens)

def count_tokens_with_multiple_tokenizers(text: str, tokenizer_names: Optional[List[str]] = None) -> Dict[str, int]:
    """
    Count tokens in text using multiple tokenizers

    Args:
        text: Text to tokenize
        tokenizer_names: List of tokenizer names to use

    Returns:
        Dictionary mapping tokenizer names to token counts
    """
    if not tokenizer_names:
        # Default tokenizers
        tokenizer_names = ["gpt2", "t5", "bert", "roberta"]

    results = {}

    for tokenizer_name in tokenizer_names:
        token_count = count_tokens(text, tokenizer_name)
        results[tokenizer_name] = token_count

    return results

def find_pdf_files():
    """Find PDF files in the current directory and subdirectories"""
    pdf_files = []
    for root, _, files in os.walk('.'):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

# For use in Colab notebooks
def analyze_pdf(pdf_path=None, tokenizers=None, custom_tokenizer=None):
    """Analyze a PDF file and count tokens"""

    if pdf_path is None:
        # List available PDF files
        pdf_files = find_pdf_files()

        if not pdf_files:
            print("No PDF files found in the current directory.")
            print("Please upload a PDF file using the code below:")
            print("from google.colab import files")
            print("uploaded = files.upload()")
            return

        print("Available PDF files:")
        for i, file in enumerate(pdf_files):
            print(f"{i+1}. {file}")

        choice = input("Enter the number of the PDF to analyze (or upload a new one): ")

        try:
            choice_idx = int(choice) - 1
            if 0 <= choice_idx < len(pdf_files):
                pdf_path = pdf_files[choice_idx]
            else:
                print("Invalid choice. Please upload a PDF file.")
                uploaded = files.upload()
                pdf_path = list(uploaded.keys())[0]
        except:
            print("Invalid input. Please upload a PDF file.")
            uploaded = files.upload()
            pdf_path = list(uploaded.keys())[0]

    # Check if the PDF file exists
    if not os.path.isfile(pdf_path):
        print(f"Error: PDF file {pdf_path} not found")
        return

    # Set default tokenizers if not provided
    if tokenizers is None:
        tokenizers = ["gpt2", "t5", "bert", "roberta"]

    # Add custom tokenizer if provided
    tokenizers_to_use = tokenizers.copy()
    if custom_tokenizer:
        tokenizers_to_use.append(custom_tokenizer)

    # Extract text from PDF
    print(f"Extracting text from {pdf_path}...")
    text = extract_text_from_pdf(pdf_path)

    if not text:
        print("No text extracted from PDF. Check if the PDF contains extractable text.")
        return

    # Print text statistics
    print(f"\nText statistics:")
    print(f"Characters: {len(text):,}")
    print(f"Words: {len(text.split()):,}")

    # Count tokens using different tokenizers
    print(f"\nCounting tokens using {len(tokenizers_to_use)} tokenizers...")
    token_counts = count_tokens_with_multiple_tokenizers(text, tokenizers_to_use)

    # Print results
    print("\nToken counts:")
    for tokenizer_name, count in token_counts.items():
        if count >= 0:
            print(f"{tokenizer_name}: {count:,} tokens")
        else:
            print(f"{tokenizer_name}: Error loading tokenizer")

    return {
        "text": text,
        "statistics": {
            "characters": len(text),
            "words": len(text.split())
        },
        "token_counts": token_counts
    }

# For direct execution in Colab
if __name__ == "__main__":
    # Filter out Colab-specific arguments
    args = [arg for arg in sys.argv if not arg.startswith('-f')]
    sys.argv = args

    # Create argparse without the required argument, which causes problems in Colab
    parser = argparse.ArgumentParser(description="Count tokens in PDF files using various tokenizers")
    parser.add_argument("pdf_path", nargs='?', help="Path to the PDF file")
    parser.add_argument("--tokenizers", nargs="+", default=["gpt2", "t5", "bert", "roberta"],
                      help="List of tokenizers to use (default: gpt2, t5, bert, roberta)")
    parser.add_argument("--custom", help="Use a custom tokenizer from Hugging Face (e.g., 'gpt-neox-20b')")

    try:
        args = parser.parse_args()
        analyze_pdf(args.pdf_path, args.tokenizers, args.custom)
    except SystemExit:
        # Called from a notebook without arguments
        analyze_pdf()

usage: colab_kernel_launcher.py [-h] [--tokenizers TOKENIZERS [TOKENIZERS ...]] [--custom CUSTOM]
                                pdf_path
colab_kernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
